In [ ]:
# %% [code]
!pip install -U tensorflow_datasets
!apt install -y fonts-nanum fonts-nanum-coding

import sys
import os
import math

import numpy as np
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras

import pathlib

In [ ]:
TAKE = 10
BATCH = 32
EPOCH = 100
REPEATS = 5

STRING_CODEC = 'UTF-8'

IMG_HEIGHT = 224
IMG_WIDTH = 224
TEXT_LEN = 64
TOKEN_LEN = 16
LATENT = 256

FOLDER_BASE = '/kaggle/input/'
FOLDER = FOLDER_BASE+'naver-posts/'
FOLDER_IMG = FOLDER + 'img/'
FOLDER_SUMMARY = FOLDER + 'summary/'
FOLDER_TEXT = FOLDER + 'text/'
VOCAB_PATH = FOLDER + 'vocab'

FOLDER_W2T2V = FOLDER_BASE+'word2token2vec/'

TOKEN_SOS = '<SOS>'
TOKEN_EOS = '<EOS>'

# Any results you write to the current directory are saved as output.
print(tf.version.VERSION)

In [ ]:
data_dir = tf.keras.utils.get_file(origin='https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
                                         fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
image_count

In [ ]:
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

In [ ]:
image_generator = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = list(CLASS_NAMES))

In [ ]:
import matplotlib.pyplot as plt

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')
        
image_batch, label_batch = next(train_data_gen)
show_batch(image_batch, label_batch)

In [ ]:
def gen():
    yield from train_data_gen

ds = tf.data.Dataset.from_generator(gen, 
                                   (tf.float32, tf.int64),
                                   (tf.TensorShape([None, IMG_HEIGHT, IMG_WIDTH, 3]), tf.TensorShape([None, len(CLASS_NAMES)])))

for x in ds.take(1):
    print(x)

In [ ]:
def model_img_encoder(img_input, latent_unit):
    def res_block(input_layer, filters, downsample=False, name=None):
        with tf.name_scope("res_block"):
            out = input_layer

            if downsample:
                out = keras.layers.Conv2D(filters * 4, (3, 3), strides=(2, 2), name=name+"_downsample")(out)
                input_layer = out
                
            out = keras.layers.Conv2D(filters, (1, 1), padding='same')(out)
            out = keras.layers.BatchNormalization()(out)
            out = keras.layers.LeakyReLU(0.2)(out)

            out = keras.layers.Conv2D(filters, (3, 3), padding='same')(out)
            out = keras.layers.BatchNormalization()(out)
            out = keras.layers.LeakyReLU(0.2)(out)

            out = keras.layers.Conv2D(filters * 4, (1, 1), padding='same')(out)  
            out = keras.layers.BatchNormalization()(out)  

            out = out + input_layer
            out = keras.layers.LeakyReLU(0.2, name=name+"_out")(out)

            return out
    
    img = img_input
    
    img = keras.layers.Conv2D(64, (7, 7), strides=(2, 2))(img)
    img = keras.layers.BatchNormalization()(img)
    img = keras.layers.LeakyReLU(0.2)(img)
    img = keras.layers.MaxPool2D((3, 3), strides=(2, 2), name="pre")(img)
    
    img = res_block(img, 64, True, name="stg1-d")
    for i in range(2):
        img = res_block(img, 64, name="stg1-"+str(i))
    
    img = res_block(img, 128, True, name="stg2-d")
    for i in range(3):
        img = res_block(img, 128, name="stg2-"+str(i))

    img = res_block(img, 256, True, name="stg3-d")
    for i in range(22):
        img = res_block(img, 256, name="stg3-"+str(i))
    
    img = res_block(img, 512, True, name="stg4-d")
    for i in range(2):
        img = res_block(img, 512, name="stg4-"+str(i))
    
    img = keras.layers.AveragePooling2D(pool_size=(2, 2), padding='same')(img)
    img = keras.layers.Flatten()(img)
    img = keras.layers.Dense(latent_unit)(img)
    return keras.Model(inputs=[img_input], outputs=[img], name='image_latent')

In [ ]:
input_image = keras.Input((IMG_HEIGHT, IMG_WIDTH, 3), name='input_img')

model_infer = model_img_encoder(input_image, LATENT)
model_infer.summary()

In [ ]:
img_latent = model_infer(input_image)

classify = keras.layers.Dense(len(CLASS_NAMES), activation='softmax')(img_latent)

model_train = keras.Model(inputs=[input_image], outputs=[classify])
model_train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model_train.summary()

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('model_image2vec_train.h5', save_best_only=True)

VALID_TAKE = 3
ds_validation = ds.take(VALID_TAKE).cache()
    
model_train.fit(ds.skip(VALID_TAKE).prefetch(TAKE*2).take(TAKE),
          validation_data=ds_validation,
          epochs=EPOCH,
          callbacks=[checkpoint])